In [ ]:
import xarray as xr
import pandas as pd
import numpy as np
import cftime
import glob
import os
from IPython.display import clear_output

# location of this script: /glade/work/sglanvil/CCR/

baseDir = "/glade/campaign/cesm/development/espwg/CESM2-DP/timeseries/"
iyear = 1999
for imbr in range(21, 31):
    member_str = f"{imbr:03d}"
    member_dir = f"{baseDir}b.e21.BSMYLE.f09_g17.{iyear}-11.{member_str}/atm/proc/tseries/day_1/"
    nc_files = glob.glob(os.path.join(member_dir, "*.nc"))
    clear_output(wait=True)  # Clears the current output
    print(f"Processing mbr: {imbr}")
    for nc_file in nc_files:
        print(nc_file)
        ds = xr.open_dataset(nc_file)
        filename = os.path.basename(nc_file)
        fileOut = f"/glade/work/sglanvil/CCR/yeager/fixed/fixed_{filename}"
        time_length = len(ds['time'])
        if time_length < 2982:
            time_units = ds['time'].encoding.get('units', None)
            time_vals = ds['time'].values
            time_strings = [x.strftime('%Y-%m-%d %H:%M:%S') for x in time_vals]
            current_time_range = pd.to_datetime(time_strings)
            start_date = current_time_range.min()
            end_date = current_time_range.max()
            full_time_range = pd.date_range(start=start_date,
                                            end=end_date, freq="D")
            full_time_range = full_time_range[~((full_time_range.month == 2)
                                                & (full_time_range.day == 29))]
            full_time_range_cftime = np.array([
                cftime.DatetimeNoLeap(x.year, x.month, x.day)
                for x in full_time_range])
            ds = ds.drop_vars('time_bnds', errors='ignore')
            ds_filled = ds.interp(time=full_time_range_cftime)
            time_numeric = cftime.date2num(full_time_range_cftime,
                                           time_units[0:-9])
            time_bnds = np.column_stack((time_numeric - 1, time_numeric))
            ds_filled['time_bnds'] = (('time', 'nbnd'), time_bnds)
            ds_filled['time_bnds'].attrs['long_name'] = "time interval endpoints"
            ds_filled['time'].encoding['units'] = str(time_units)  # eg, 'days since 1999-11-01 00:00:00'
            ds_filled.to_netcdf(fileOut)


Processing mbr: 21
/glade/campaign/cesm/development/espwg/CESM2-DP/timeseries/b.e21.BSMYLE.f09_g17.1999-11.021/atm/proc/tseries/day_1/b.e21.BSMYLE.f09_g17.1999-11.021.cam.h1.Q050.20011031-20091231.nc
/glade/campaign/cesm/development/espwg/CESM2-DP/timeseries/b.e21.BSMYLE.f09_g17.1999-11.021/atm/proc/tseries/day_1/b.e21.BSMYLE.f09_g17.1999-11.021.cam.h1.FLNR.20011031-20091231.nc
/glade/campaign/cesm/development/espwg/CESM2-DP/timeseries/b.e21.BSMYLE.f09_g17.1999-11.021/atm/proc/tseries/day_1/b.e21.BSMYLE.f09_g17.1999-11.021.cam.h1.FSNTOAC.20011031-20091231.nc
/glade/campaign/cesm/development/espwg/CESM2-DP/timeseries/b.e21.BSMYLE.f09_g17.1999-11.021/atm/proc/tseries/day_1/b.e21.BSMYLE.f09_g17.1999-11.021.cam.h1.FLNTC.20011031-20091231.nc
/glade/campaign/cesm/development/espwg/CESM2-DP/timeseries/b.e21.BSMYLE.f09_g17.1999-11.021/atm/proc/tseries/day_1/b.e21.BSMYLE.f09_g17.1999-11.021.cam.h1.V500.20011031-20091231.nc
/glade/campaign/cesm/development/espwg/CESM2-DP/timeseries/b.e21.BSMYLE.